<a href="https://colab.research.google.com/github/Auxilus08/Compiler-Design/blob/main/Cd_practical3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parser Construction

This cell initializes the global variables `grammer`, `non_terminals`, and `first_sets` which will be used to store the grammar rules, the set of non-terminal symbols, and the computed FIRST sets, respectively.

In [72]:
import sys
grammer = {}

non_terminals = set()

first_sets = {}

This cell defines the `parse_grammer` function, which takes a list of input lines representing grammar rules, clears the existing grammar and non-terminals, and then parses each line to populate the `grammer` dictionary and `non_terminals` set. It expects rules in the format `NonTerminal -> Production1 | Production2 | ...`.

In [73]:
def parse_grammer(input_lines):
  grammer.clear()
  non_terminals.clear()

  for line in input_lines:
    line = line.strip()
    if not line or line.startswith('#'):
      continue
    head, _, str_productions = line.partition(':')
    head = head.strip()

    if not head.isupper():
      print(f" Non terminal {head} is not uppercase.")

    non_terminals.add(head)

    if head not in grammer:
      grammer[head] = []

    productions = str_productions.split('|')
    for production in productions:
      production = production.strip()
      grammer[head].append(production.split()) # Split production into symbols

This cell defines the `calculate_first_for_sequence` function, which computes the FIRST set for a given sequence of grammar symbols (terminals and non-terminals). It handles Epsilon productions and considers the FIRST sets of individual symbols in the sequence.

In [74]:
def calculate_first_for_sequence(sequence):
  first = set()

  if not sequence or (len(sequence) == 1 and sequence[0] == 'Epsilon'): # Handle single 'Epsilon'
    return {'Epsilon'}

  can_produce_epsilon = True

  for symbol in sequence:
    if symbol not in non_terminals:
      first.add(symbol)
      can_produce_epsilon = False
      break

    first_of_symbol = first_sets.get(symbol, set()) # Use .get() with empty set default
    first.update(first_of_symbol - {'Epsilon'})

    if 'Epsilon' not in first_of_symbol:
      can_produce_epsilon = False
      break

  if can_produce_epsilon:
    first.add('Epsilon')

  return first


This cell defines the `compute_first_set` function, which iteratively calculates the FIRST sets for all non-terminals in the grammar until no further changes occur.

In [75]:
def compute_first_set():

  for nt in non_terminals:
    first_sets[nt] = set()

  while True:
    changed = False

    for nt in non_terminals:
      original_size = len(first_sets[nt])

      for production in grammer.get(nt, []):
        first_of_rhs = calculate_first_for_sequence(production)
        first_sets[nt].update(first_of_rhs)

      if len(first_sets[nt]) != original_size:
        changed = True

    if not changed:
      break


In [76]:
def main():

  print("Enter the Grammer Rules: ")
  print("use 'Epsilon'. press Enter on an Empty line when you are done")
  print("-" * 20)

  input_lines = []
  while True:
    try:
      line = input()
      if not line:
        break
      input_lines.append(line)

    except EOFError:
      break

  if not input_lines:
    print("No grammer rules for Entered. Exiting")
    return

  try:

    parse_grammer(input_lines)

    compute_first_set()

    print("\nFIRST Sets:")
    print("-" * 20)
    sorted_nts = sorted(list(non_terminals)) # Sort non-terminals for consistent output order
    for nt in sorted_nts:

            terminals_str = ", ".join(sorted(list(first_sets.get(nt, set()))))
            print(f"FIRST({nt}) = {{ {terminals_str} }}") # Print the FIRST set

  except Exception as e:
        print(f"\nAn error occurred: {e}", file=sys.stderr)

In [77]:
if __name__ == "__main__":
    main()

Enter the Grammer Rules: 
use 'Epsilon'. press Enter on an Empty line when you are done
--------------------
A : S B | B
S : a | B c | Epsilon
B : b | d


FIRST Sets:
--------------------
FIRST(A) = { a, b, d }
FIRST(B) = { b, d }
FIRST(S) = { Epsilon, a, b, d }


# Task
Fix the provided Python code to correctly calculate the FOLLOW sets and construct the LL(1) parsing table for the given grammar, using the previously computed FIRST sets. Display the calculated FOLLOW sets and the resulting LL(1) parsing table.

## Compute follow sets

### Subtask:

Define a function to compute the FOLLOW sets for each non-terminal using the previously computed FIRST sets and the grammar rules.

In [78]:
def compute_follow_set():

  # Initialize FOLLOW sets for all non-terminals as empty sets
  follow_sets = {nt: set() for nt in non_terminals}

  # Rule 1: Add '$' to the FOLLOW set of the start symbol.
  # Assuming the first non-terminal in the grammer dictionary is the start symbol.
  start_symbol = list(grammer.keys())[0]
  follow_sets[start_symbol].add('$')


  changed = True
  while changed:
    changed = False


    for head, productions in grammer.items():
      for production in productions:
        for i, symbol in enumerate(production):
          if symbol in non_terminals:
            # Get the sequence of symbols after the current non-terminal
            beta = production[i+1:]
            # Calculate the FIRST set of the sequence beta
            first_of_beta = calculate_first_for_sequence(beta)

            original_size = len(follow_sets[symbol])

            # Rule 2: If beta is not empty and does not produce Epsilon,
            # add FIRST(beta) (excluding Epsilon) to FOLLOW(symbol).
            follow_sets[symbol].update(first_of_beta - {'Epsilon'})

            # Rule 3: If beta is empty or produces Epsilon,
            # add FOLLOW(head) to FOLLOW(symbol).
            if 'Epsilon' in first_of_beta or not beta:
              follow_sets[symbol].update(follow_sets[head])

            # Check if the size of the FOLLOW set changed after processing this rule
            if len(follow_sets[symbol]) != original_size:
              changed = True # If the size changed, set the changed flag to True
  return follow_sets # Return the computed FOLLOW sets

## Construct ll(1) parsing table

### Subtask:
Define a function to construct the LL(1) parsing table using the computed FIRST and FOLLOW sets.


In [81]:
def construct_ll1_table(grammer, first_sets, follow_sets):
  parsing_table = {}

  terminals = set()
  for productions in grammer.values():
    for production in productions:
      for symbol in production:
        if symbol not in non_terminals and symbol != 'Epsilon':
          terminals.add(symbol)
  terminals.add('$') # Add the end-of-input marker to the terminals

  for nt in non_terminals:
    parsing_table[nt] = {}
    for t in terminals:
      parsing_table[nt][t] = None

  for head, productions in grammer.items():
    for production in productions:
      # Calculate the FIRST set of the current production
      first_of_rhs = calculate_first_for_sequence(production)

      # Rule 1: If Epsilon is in FIRST(production), for each terminal 'a' in FOLLOW(head),
      # add the production (head -> production) to M[head, a].
      if 'Epsilon' in first_of_rhs:
        for terminal in follow_sets.get(head, set()):
           # Check for conflicts (multiple productions for the same table entry)
           if parsing_table[head][terminal] is not None:
             print(f"Conflict at M[{head}, {terminal}]: attempting to add {production} but {parsing_table[head][terminal]} already exists.")
           parsing_table[head][terminal] = production
      else:
        # Rule 2: For each terminal 'a' in FIRST(production) (where 'a' is not Epsilon),
        # add the production (head -> production) to M[head, a].
        for terminal in first_of_rhs:
          if terminal != 'Epsilon':
            # Check for conflicts
            if parsing_table[head][terminal] is not None:
              print(f"Conflict at M[{head}, {terminal}]: attempting to add {production} but {parsing_table[head][terminal]} already exists.")
            parsing_table[head][terminal] = production

  return parsing_table

## Present results

### Subtask:
Display the computed FOLLOW sets and the constructed LL(1) parsing table.


In [83]:

follow_sets = compute_follow_set()
parsing_table = construct_ll1_table(grammer, first_sets, follow_sets)

print("\nFOLLOW Sets:")
print("-" * 20)
sorted_nts = sorted(list(non_terminals)) # Sort non-terminals for consistent output order
for nt in sorted_nts:
    # Get the FOLLOW set, sort it, and join elements with commas
    terminals_str = ", ".join(sorted(list(follow_sets.get(nt, set()))))
    print(f"FOLLOW({nt}) = {{ {terminals_str} }}")

# Display the constructed LL(1) Parsing Table
print("\nLL(1) Parsing Table:")
print("-" * 20)

# Determine all terminals including the end-of-input marker for table headers
terminals = set()
for productions in grammer.values():
    for production in productions:
        for symbol in production:
            if symbol not in non_terminals and symbol != 'Epsilon':
                terminals.add(symbol)
terminals.add('$')
sorted_terminals = sorted(list(terminals))

header = "{:<10}".format("Non-Terminal")
for t in sorted_terminals:
    header += "{:<10}".format(t)
print(header)
print("-" * (10 * (len(sorted_terminals) + 1)))


for nt in sorted_nts:
    row = "{:<10}".format(nt)
    for t in sorted_terminals:
        production = parsing_table.get(nt, {}).get(t)
        if production is not None:
            production_str = " ".join(production)
            row += "{:<10}".format(f"{nt} -> {production_str}")
        else:
            row += "{:<10}".format("")
    print(row)

Conflict at M[A, d]: attempting to add ['B'] but ['S', 'B'] already exists.
Conflict at M[A, b]: attempting to add ['B'] but ['S', 'B'] already exists.
Conflict at M[S, d]: attempting to add ['Epsilon'] but ['B', 'c'] already exists.
Conflict at M[S, b]: attempting to add ['Epsilon'] but ['B', 'c'] already exists.

FOLLOW Sets:
--------------------
FOLLOW(A) = { $ }
FOLLOW(B) = { $, c }
FOLLOW(S) = { b, d }

LL(1) Parsing Table:
--------------------
Non-Terminal$         a         b         c         d         
------------------------------------------------------------
A                   A -> S B  A -> B              A -> B    
B                             B -> b              B -> d    
S                   S -> a    S -> Epsilon          S -> Epsilon
